# Wan2GP on Google Colab

Set up [Wan2GP](https://github.com/deepbeepmeep/Wan2GP) in a fresh GPU-backed Colab session. Run the cells in order to prepare the runtime, install dependencies, and launch the Gradio interface.


## 1. Confirm the accelerator

Choose `Runtime → Change runtime type` and select **GPU** before running anything else.

If this cell raises an error, go back to `Runtime → Change runtime type`, pick **GPU** or **TPU** and save.


In [ ]:
import subprocess

try:
    subprocess.run(['nvidia-smi'], check=True)
except Exception as exc:
    raise RuntimeError(
        'GPU not detected. In Colab, open Runtime → Change runtime type, select GPU (or TPU if GPUs are unavailable), save, then rerun this cell.'
    ) from exc


## 2. Configure the workspace path

Choose where Wan2GP should be installed. Update `WAN2GP_ROOT` if you prefer a different location.


In [ ]:
from pathlib import Path

WAN2GP_ROOT = Path('/content/Wan2GP').resolve()
print(f'Wan2GP will be installed to: {WAN2GP_ROOT}')


## 3. Download or update Wan2GP

Clone the repository if it is not present yet; otherwise pull the latest changes.


In [ ]:
import subprocess

repo_url = 'https://github.com/deepbeepmeep/Wan2GP.git'
if WAN2GP_ROOT.exists():
    print('Repository already exists. Pulling latest changes...')
    subprocess.run(['git', '-C', str(WAN2GP_ROOT), 'pull'], check=True)
else:
    subprocess.run(['git', 'clone', repo_url, str(WAN2GP_ROOT)], check=True)


## 4. Install system dependencies

Install shared libraries needed for video and audio processing. If you see a warning about skipping an extra repository (for example `r2u.stat.illinois.edu`), it is safe to ignore.

In [ ]:
import os, subprocess

env = os.environ.copy()
env['DEBIAN_FRONTEND'] = 'noninteractive'

subprocess.run(['sudo', 'apt-get', 'update', '-qq'], check=True, env=env)
subprocess.run([
    'sudo', 'apt-get', 'install', '-y', '--no-install-recommends',
    'ffmpeg', 'libglib2.0-0', 'libgl1', 'libportaudio2'
], check=True, env=env)


## 5. Install Python dependencies

Install PyTorch and Wan2GP's Python packages. Colab's GPU runtimes currently ship with CUDA 12.1 drivers, so the commands below pull the matching PyTorch wheels; adjust only if Google upgrades the runtime.


In [ ]:
import os, subprocess, sys

env = os.environ.copy()
env.setdefault('DEBIAN_FRONTEND', 'noninteractive')

subprocess.run([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip', 'setuptools', 'wheel'], check=True, env=env)
subprocess.run([sys.executable, '-m', 'pip', 'install', 'torch==2.8.0', 'torchvision', 'torchaudio', '--index-url', 'https://download.pytorch.org/whl/cu121'], check=True, env=env)
subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', str(WAN2GP_ROOT / 'requirements.txt')], check=True, env=env)
subprocess.run([sys.executable, '-m', 'pip', 'install', 'gradio==5.35.0', 'sageattention==1.0.6'], check=True, env=env)


## 6. Launch Wan2GP

Run the Gradio interface. The first launch downloads large model files, so expect a long wait. Keep the cell running to stay connected; stop it with the square **Stop** button when you are finished. Try `--profile 5` if you see GPU memory errors.


In [ ]:
import os, subprocess, sys

env = os.environ.copy()
env.setdefault('WAN_CACHE_DIR', str(WAN2GP_ROOT / 'models'))
cmd = [
    sys.executable,
    '-u',
    'wgp.py',
    '--listen',
    '--server-port', '7860',
    '--share',
    '--profile', '4',
]
print('Launching Wan2GP…')
process = subprocess.Popen(
    cmd,
    cwd=str(WAN2GP_ROOT),
    env=env,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
)
try:
    for line in process.stdout:
        print(line, end='')
except KeyboardInterrupt:
    print('Stopping Wan2GP…')
    process.terminate()
finally:
    process.wait()
